# STAC Extensions

Here we explore different extensions for out dataset.

In [103]:
import pystac

eurosat = pystac.Catalog.from_file('eurosat-stac/catalog.json')
eurosat

ID: eurosat
Description: EuroSAT dataset
type: Catalog
ID: sentinel2
Description: EuroSAT Sentinel 2 dataset
type: Collection
stac_extensions: []
ID: Pasture_354
"Bounding Box: [557139.766364, 5882832.541223793, 557778.4111188718, 5883473.46161]"
Datetime: 2015-10-22 00:00:00+00:00
datetime: 2015-10-22T00:00:00Z


The EO extension improves the metadata of our items including information such as the bands, cloud cover, ...

https://github.com/stac-extensions/eo

https://pystac.readthedocs.io/en/stable/tutorials/how-to-create-stac-catalogs.html#Creating-an-Item-that-implements-the-EO-extension

In [104]:
from pystac.extensions.eo import Band, EOExtension

s2_bands = [
	Band.create(name='Aerosols', description='Coastal aerosol, 442.7 nm (S2A), 442.3 nm (S2B)', common_name='coastal'),
	Band.create(name='Blue', description='Blue, 492.4 nm (S2A), 492.1 nm (S2B)', common_name='blue'),
	Band.create(name='Green', description='Green, 559.8 nm (S2A), 559.0 nm (S2B)', common_name='green'),
	Band.create(name='Red', description='Red, 664.6 nm (S2A), 665.0 nm (S2B)', common_name='red'),
	Band.create(name='Red edge 1', description='Vegetation red edge, 704.1 nm (S2A), 703.8 nm (S2B)', common_name='rededge'),
	Band.create(name='Red edge 2', description='Vegetation red edge, 740.5 nm (S2A), 739.1 nm (S2B)', common_name='rededge'),
	Band.create(name='Red edge 3', description='Vegetation red edge, 782.8 nm (S2A), 779.7 nm (S2B)', common_name='rededge'),
	Band.create(name='NIR', description='NIR, 832.8 nm (S2A), 833.0 nm (S2B)', common_name='nir'),
	Band.create(name='Red edge 4', description='Narrow NIR, 864.7 nm (S2A), 864.0 nm (S2B)', common_name='nir08'),
	Band.create(name='Water vapour', description='Water vapour, 945.1 nm (S2A), 943.2 nm (S2B)', common_name='nir09'),
	Band.create(name='Cirrus', description='SWIR – Cirrus, 1373.5 nm (S2A), 1376.9 nm (S2B)', common_name='cirrus'),
	Band.create(name='SWIR1', description='SWIR, 1613.7 nm (S2A), 1610.4 nm (S2B)', common_name='swir16'),
	Band.create(name='SWIR2', description='SWIR, 2202.4 nm (S2A), 2185.7 nm (S2B)', common_name='swir22'),
]

In [113]:
from tqdm import tqdm 

for item in tqdm(eurosat.get_all_items()):
	eo = EOExtension.ext(item, add_if_missing=True) # not adding in stac_extensions...
	eo.apply(bands=s2_bands)
	# item.validate() # error because of missing stac_extensions

100it [00:00, 1167.57it/s]


The label extension adds information about the labels of our dataset, making it ML ready.

https://github.com/stac-extensions/label

https://pystac.readthedocs.io/en/stable/tutorials/pystac-spacenet-tutorial.html#Create-new-STAC-with-COGs-and-labels

In [99]:
from pystac.extensions.label import LabelClasses, LabelExtension, LabelType
import os 
from pathlib import Path

path = Path('data')
cats = sorted(os.listdir(path / 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif'))

for item in tqdm(eurosat.get_all_items()):
	label = LabelExtension.ext(item, add_if_missing=True) # not adding in stac_extensions...
	label.apply(
		label_description=f'label for scene {item.id}',
		label_type=LabelType.VECTOR,
		label_properties=[item.id.split('_')[0]],
		label_classes = [LabelClasses.create(classes=cats, name="classes")]
	)

100it [00:00, 34225.25it/s]


In [100]:
# eurosat.validate_all() # error because of missing stac_extensions
eurosat.normalize_and_save(root_href="eurosat-stac-ext", catalog_type=pystac.CatalogType.SELF_CONTAINED)

See all the available extensions at https://stac-extensions.github.io/

Doubts:

- Extensions apply only on items? Can we have additional metadata at the collection or catalog level?
- Can we update the stac_extensions property of an item? 
- The label must be a vector or raster, not option for a class?